## OSI
The OSI Model is a `logical and conceptual model` that defines network communication used by systems open to interconnection and communication with other systems. The `Open System Interconnection (OSI Model)` also defines a logical network and effectively describes computer packet transfer by using various layers of protocols.

The OSI Model can be seen as a universal language for computer networking. It's based on the concept of splitting up a communication system into seven abstract layers, each one stacked upon the last.

### OIS model leayers:
1. Application Layer (Layer 7): Provides network services directly to end-users; interfaces with software applications to implement a communicating component.
   - Network resource sharing, remote file access, network management.
   - Examples: HTTP (HyperText Transfer Protocol), FTP (File Transfer Protocol), SMTP (Simple Mail Transfer Protocol), DNS (Domain Name System).

2. Presentation Layer (Layer 6):
   - Function: Translates data between the application layer and the network; handles data encryption and decryption, compression, and translation.
   - Examples: SSL/TLS (Secure Sockets Layer / Transport Layer Security), JPEG, MPEG.

3. Session Layer (Layer 5):
   - Function: Manages sessions and controls dialogues between computers; establishes, maintains, and terminates connections.
   - Examples: NetBIOS, PPTP (Point-to-Point Tunneling Protocol).

4. Transport Layer (Layer 4):
   - Function: Provides reliable data transfer services to the upper layers; handles error recovery and flow control.
   - Examples: TCP (Transmission Control Protocol), UDP (User Datagram Protocol).

5. Network Layer (Layer 3):
   - Function: Determines the best physical path for data to reach its destination; handles packet forwarding and routing.
   - Examples: IP (Internet Protocol), Routers, ICMP (Internet Control Message Protocol).

6. Data Link Layer (Layer 2):
   - Function: Provides node-to-node data transfer and handles error detection and correction from the physical layer.
   - Examples: MAC addresses, Ethernet, Switches, Bridges, PPP (Point-to-Point Protocol).

7. Physical Layer (Layer 1):
   - Function: Transmits raw bit streams over a physical medium.
   - Examples: Cables (Ethernet cables, fiber optics), Hubs, Network Interface Cards (NICs).

### Hypertext transfer protocol (HTTP)
HTTP is a method for `encoding and transporting` data between a client and a server. It is a `request/response` protocol: clients issue requests and servers issue responses with relevant content and completion status info about the request. HTTP is self-contained, allowing requests and responses to flow through many intermediate routers and servers that perform load balancing, caching, encryption, and compression.

### TCP vs UDP:
1. TCP: TCP is a connection-oriented protocol that ensures reliable and ordered delivery of data between applications over a network. It establishes a connection before transmitting data and ensures that data packets are delivered in the same order in which they were sent, with no loss or duplication.
   - Web browsing (HTTP/HTTPS): Browsers use TCP to load web pages reliably.
   - Email (SMTP, IMAP, POP3): Email protocols use TCP to ensure messages are delivered correctly.
   - File transfer (FTP): FTP uses TCP to reliably transfer files between computers.
   - Remote access (SSH, Telnet): Secure remote access protocols use TCP to maintain reliable connections.


2. UDP: UDP is a connectionless protocol that allows applications to send messages, called datagrams, without establishing a connection. It provides a simple and fast transmission mechanism but does not guarantee reliability, order, or error correction.
- streaming media, online game, DNS.

## Domain Name System (DNS)
A Domain Name System (DNS) translates a domain name such as `www.example.com` to an IP address.

### How DNS works?
DNS lookup involves the following eight steps:

1. A client types example.com into a web browser, the query travels to the internet and is received by a DNS resolver.
2. The resolver then recursively queries a DNS root nameserver.
3. The root server responds to the resolver with the address of a Top-Level Domain (TLD).
4. The resolver then makes a request to the .com TLD.
5. The TLD server then responds with the IP address of the domain's nameserver, example.com.
6. Lastly, the recursive resolver sends a query to the domain's nameserver.
7. The IP address for example.com is then returned to the resolver from the nameserver.
8. The DNS resolver then responds to the web browser with the IP address of the domain requested initially.

## Microservice Communication:


### Sync Communication:
Basically, we can say that Synchronous communication is using HTTP or gRPC protocol for returning sync response. The client sends a request and waits for a response from the service. So that means client code block their thread, until the response reach from the server.

- Use gRPC

### Async Communication:
Basically, In Asynchronous communication, the client sends a request but it doesn’t wait for a response from the service. So the key point here is that, the client should not have blocked a thread while waiting for a response.

The most popular protocol for this Asynchronous communications is AMQP (Advanced Message Queuing Protocol). So with using AMQP protocols, the client sends the message with using message broker systems like Kafka and RabbitMQ queue.

An asynchronous systems can be implemented in a one-to-one(queue) mode or one-to-many (topic) mode. Basically an event-bus or message broker system is publishing events between multiple microservices, and communication provide with subscribing these events in an async way.

### Request-Response:
A microservice dispatches a request to another microservice, seeking the execution of a task, and anticipates receiving a response detailing the outcome. These can be achieved using RPC protocols like REST, gRPC, and GraphQL. Also, asynchronous way for request-response can be achieved using push notifications using web sockets or signalr, pull (polling) between the services and queue based messaged.
- gRPC

### Event-Driven Communication:
Event-Driven Communication is a variant of asynchronous communication, where microservices communicate through a message broker that facilitates the exchange of events. Each microservice publishes events when certain actions occur, and other microservices can subscribe to those events and react accordingly. Event-driven communication can enable loosely-coupled systems and provide greater flexibility, but it can also introduce potential issues with event ordering and consistency.

## API gateway:
The API Gateway is an API management tool that sits between a client and a collection of backend services. It is a single entry point into a system that encapsulates the internal system architecture and provides an API that is tailored to each client. It also has other responsibilities such as authentication, monitoring, load balancing, caching, throttling, logging, etc.

Features:
1. Authentication and Authorization
2. Service discovery
3. Reverse Proxy
4. Caching
5. Security
6. Retry and Circuit breaking
7. Load balancing
8. Logging, Tracing
9. API composition
10. Rate limiting and throttling
11. Versioning
12. Routing
13. IP whitelisting or blacklisting

### 2#Service Discovery:
Service discovery is a mechanism that allows services in a distributed system to find and communicate with each other dynamically. It hides the complex details of where services are located, so they can interact without knowing each other's exact network spots.
- Pros: Dynamic discovery, load balancing.
- Cons: Added complexity in managing the registry.

#### Client Side Discovery:

#### Server Side Discovery:
In server-side discovery, the service consumer makes a request to an intermediary `(load balancer or an API gateway)`, which then queries the service registry and routes the request to an appropriate service instance.
- Pros:
- Cons:

#### Service mesh:
Service-to-service communication is essential in a distributed application but routing this communication, both `within and across application clusters`, becomes increasingly complex as the number of services grows. Service mesh `enables managed, observable, and secure communication` between individual services. It works with a `service discovery protocol` to `detect services`. `Istio and envoy` are some of the most commonly used service mesh technologies.


#### Apache Zookeeper:
Zookeeper is a centralized service for maintaining configuration information, naming, providing distributed synchronization, and providing group services. It is often used for service discovery in distributed systems.

### 3#Proxy Server:
A proxy server acts as an intermediary between a client and a server, forwarding requests from the client to the server and returning the server's response to the client. Proxy servers are used for various purposes, including improving security, enhancing performance, and enabling access control.

1. Forward Proxy: A forward proxy acts on behalf of clients. It takes requests from clients, forwards them to external servers, and returns the responses to the clients. Clients configure their settings to use a forward proxy. Forward proxy sits between `client and internet`.
   - Content filtering
   - internet access control
   - Allows access to `geo-restricted` content
   - protect user identity online.

2. Reverse Proxy:  reverse proxy acts on behalf of servers. Clients do not need to configure anything to use a reverse proxy; they simply send requests to the proxy, which forwards them to the appropriate backend server. The reverse proxy makes the backend infrastructure more resilient and scalable.
   - Load balancing
   - SSL termination
   - hiding the identity of the backend servers(protecting the server)
   - Encrypting and decrypting SSL communications
```
Client (User)    ─────>    Nginx (Reverse Proxy)    ─────>    Backend Servers
       ^                    ^                          ^             ^
       |                    |                          |             |
  Sends request         Receives request           Forwards request  |
                       from client, forwards to   to appropriate     |
                       backend server, retrieves   backend server,   |
                       response, and sends it     retrieves response,|
                       back to the client         and sends it back  |
                                                  to Nginx           |
```


#### Why is Nginx called a “reverse” proxy?
Nginx is referred to as a "reverse proxy" because it stands in front of web servers and handles client requests on their behalf. The term "reverse proxy" distinguishes it from a "forward proxy," which acts on behalf of clients to access other servers.

1. Load Balancing: Nginx can distribute incoming requests across multiple backend servers to balance the load and improve performance and reliability. This ensures no single server becomes a bottleneck.

2. SSL Termination: Nginx can handle SSL/TLS encryption, offloading the decryption work from the backend servers. This simplifies SSL certificate management and can improve performance by reducing the load on backend servers.

3. Caching: Nginx can cache responses from backend servers and serve them to clients, reducing the load on backend servers and improving response times for clients.

4. Compression: Nginx can compress responses before sending them to clients, reducing bandwidth usage and speeding up response times.

5. Security: Nginx can `hide the details of backend servers from clients`, providing an additional layer of security. It can also be configured to `filter and block malicious traffic`.

6. URL Rewriting: Nginx can rewrite URLs in client requests before forwarding them to backend servers. This can be useful for things like redirecting requests or hiding the structure of backend URLs.

### 6#Retry and Circuit Breaker:
**1. Rettry:** The Retry pattern is a strategy where an operation is automatically retried a certain number of times before failing. It is often used in scenarios where `transient faults`, such as `temporary network issues or momentary service outages`, are expected and can resolve themselves if retried.

2. **Circuit Breaker:** The Circuit Breaker pattern is used to detect failures and prevent the application from attempting an operation that is likely to fail. It acts as a state machine with three states:
   - Closed (normal operation)
   - Open (failure detected, stop requests)
   - Half-Open (testing if the operation has recovered)
   - Example: An API gateway that calls multiple backend services. If one service is down, the Circuit Breaker prevents repeated failed calls by setting state to `Open`, protecting the system from cascading failures and improving overall stability.


#### Why do we need circuit breaker?

### 6#load Balancing:
Load balancing is a method used to distribute network or application traffic across multiple servers. This helps ensure no single server becomes overwhelmed with too much traffic, improving the overall performance, reliability, and availability of applications and websites.
- Hardware Load Balancers: Dedicated physical devices that distribute traffic across servers.
- Software Load Balancers: Software applications, such as Nginx, HAProxy, or software built into cloud services, that distribute traffic.

### 6#Logging and Tracing:
1. **Logging:** Logging involves recording information about the `application's runtime behavior`. Logs capture details about application `events, errors, transactions, and user activities`, providing a historical record of what happened.

2. **Tracing:** Tracing is the practice of tracking the `flow of requests` through a distributed system. It involves recording the path of requests as they travel through various services and components, capturing `timing information` to analyze performance and identify bottlenecks.
   - A microservices-based application where a user's request triggers calls to `multiple backend services`. Tracing allows you to follow the request through `each service and measure the time spent in each`.

### 6#Rate Limiting:
Rate limiting is a technique used to control the rate at which clients can send requests to a server. It helps protect services from being overwhelmed by too many requests in a short period and ensures fair usage among clients.
- Example: For example, rate limiting the product search API to 10 requests per minute per user to prevent excessive load and ensure fair usage.

## Web:

### URL, URI, URN
1. URL (Uniform Resource Locator): A URL is a `specific type of URI` that provides the address of a unique resource on the web.
   - `https://www.example.com:443/path/to/resource?key=value#section`
   - Scheme: Indicates the protocol to be used (e.g., `http, https, ftp, jdbc`).
   - Host: The domain name or IP address of the server (e.g., www.example.com).
   - Port (optional): The port number on the server (e.g., `:80 for HTTP`, `:443 for HTTPS`).
   - Path: The specific path to the resource on the server (e.g., /path/to/resource).
   - Query (optional): A string of key-value pairs for additional parameters (e.g., `?key=value`).
   - Fragment (optional): A reference to a specific part of the resource (e.g., `#section`).
2. URI (Uniform Resource Identifier): A URI is a generic term for a string of characters that uniquely identifies a resource on the internet. A URI can be further classified into a `URL or a URN`.
   - `https://www.example.com:443/path/to/resource?key=value#section`
   - `urn:isbn:978-3-16-148410-0`
3. URN (Uniform Resource Name): A URN is a type of URI that names a resource without specifying its location. A URN is intended to serve as a persistent, `location-independent resource identifier`.
   - Schema+name(urn: name): `urn:isbn:978-3-16-148410-0`